In [ ]:
###############
# Service version : 1.2.2
# Language        : Python/Jupyter

# Energy Loss Training
This recipe performs the training of a model that calculates an electrical transformer's the load factor, loss factor and energy loss in any given range of time. 


The data requirements are power quality data that contain timestamps, active power readings, and completion of the provided configuration file.

## Input
The data requirements for running the recipe for a given asset are Timestamps, and Active Power measurements:


| **asset_id** | **DeviceTimeStamp** | **Active Power (KW)** |
|--------------|---------------------|--------|
| 9010        | 2019-06-27T10:51    | 78.23312|
| 9010        | 2019-06-27T10:52    | 76.65175|
| 9010        | 2019-06-27T10:53    | 78.66275|
| 9010        | 2019-06-27T10:53    | 76.65175|

## Output


Load Factor: The ratio of the transformer's average load to its maximum rated capacity, indicating how efficiently it is utilized.

Loss Factor: A measure of the transformer's energy loss during operation, accounting for factors such as resistance, core, and stray load losses as per either Gustafson's or Buller & Woodrow's method.

Estimated Energy Loss: A calculated value representing the anticipated energy loss in the transformer during a given period, based on its load factor and peak power.


| asset_id | period | load_factor | loss_factor | energyloss (WH) |
|-------------|-------------|-------------|-------------|-------------|
|9010| 2019-06-25  | 0.353592    | 0.137005    | 345.253615 |
|9010| 2019-07-25  | 0.471900    | 0.237904    | 599.518190 |
|9010| 2019-08-24  | 0.512072    | 0.278126    | 700.876400 |
|9010| 2019-09-23  | 0.525415    | 0.292146    | 736.208374 |

# Import Libraries & Functions

In [ ]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
# Print functions
def print_job_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    if "status" in status_data:
        print("The status of job {} is {}.".format(job_id, status_data['status']))
    else:
        print(status_data)
    
def print_all_status(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    status_data = get_response.json()
    
    # Print the job status
    print(status_data)


In [ ]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))
    return summary_data

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            if "status" in summary:
                html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code


# Loading  Dataset

Loading the power quality dataset from the local project space

In [ ]:
df = pd.read_csv("../data/training.csv")
df["Timestamp"] = df["Timestamp"].astype(str)
df.head(10)

# Energy Loss Deployment

## Defining Paths

In [ ]:
# Define the file paths
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
config_file_path = "../config/parameter_config_three_kpis_loss_rate.yaml"
data_file_path = "../data/training.csv"

## Preparation of Configuration File

A requirement of the recipe is preparation of the configuration file for training. 


Information including column specification, loss factor calculation method, the option to use an a priori peak power are all options that can be configured.


Most important the "output" can be changed to "energy_loss", "load_factor" or "loss_factor"

For more detail for preparing the configuration file see the [documentation](https://github.ibm.com/srom/ModelFactory/blob/development/model_factory/recipes/electrical_transformer/docs/energyloss.md)

In [ ]:
import requests

# Read the content of the config file
with open(config_file_path, 'r') as config_file:
    config_content = config_file.read()

# Display the content
print(config_content)

## POST Response

In [ ]:
import requests

files = {
    "data_file": ("training.csv", open(data_file_path, 'rb')),
    "config_file": ("parameter_config.yaml", open(config_file_path, 'rb')),    
}

url = endpoint_url + "/recipe/electrical-transformer/kpi/energyloss"
post_response = requests.request("POST", url, headers={}, data={}, files=files)

In [ ]:
post_r_json = post_response.json()

anomaly_service_jobId = None

if 'jobId' in post_r_json:
    anomaly_service_jobId = post_r_json['jobId']
    print ('submitted successfully job : ', post_r_json['jobId'])
else:
    print (post_r_json)
time.sleep(5)

## GET Response  - Keep Refresh it for a while until seeing the STATUS changed to DONE

The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [ ]:
log_url = endpoint_url + "/log/"
job_id = post_r_json['job_id']

In [ ]:
print(log_url + job_id)

## GET Response

In [ ]:
job_id = post_r_json["job_id"]
# Get Status
print_job_status(job_id,endpoint_url)

The status of the job may be running, flagged by INITALIZING or EXECUTING

After a while the model recipe training is complete, and the STATUS changes to DONE

In [ ]:
# Get Status
print_job_status(job_id,endpoint_url)

In [ ]:
# Get selected details
json_data = print_job_details(job_id,endpoint_url)

# Get all details

print_all_status(job_id, endpoint_url)

### Saving details for deployment

In [ ]:
json_data["detailed_summary"]

In [ ]:
json_data["detailed_summary"]

In [ ]:
import yaml
if 'status' in json_data:
    if json_data['status'] == 'DONE':
        for summary in json_data["detailed_summary"]:
            if summary["params.output_artifact_name"] != "":
                s3_uri = summary["artifact_uri"] + "/" + summary["params.output_artifact_name"]
            if summary['tags.artifact_name']!="":
                model_uri = "runs:/" + summary['run_id'] + "/" + summary['tags.artifact_name']
        output_data = {
            "model_uri" : model_uri,
            "train_job_id" : job_id,
            "s3_uri": s3_uri,
        }
        with open("../config/model_info.yml","w") as file:
            yaml.dump(output_data, file)
        output_data

In [ ]:
output_data